# Co-factor swap targets

The co-factor balance is important for the fitness of the microorganisms, but in order to optimize the flux of heterologous pathways, it can be neccessary to swap co-factors in certain reactions. The "CofactorSwapOptimization" algorithm is a variant of Optswap, which swaps cofactors in reactions and sees what effect on the fitness function it has. In this case we define the fitness function as lycopene yield on glucose for swapping FAD with NADP, since that is the co-factor in our heterologous pathway.

In [2]:
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cobra.io import read_sbml_model
model = read_sbml_model('C:\Users\pauli\OneDrive\Dokumenty\DTU\Cell factories\group-assingment-team11\Models/yeast8.xml')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3366528614.py, line 4)

Here we define the co-factor swap algorithm and the parameters in the way described above.

In [2]:
Fluxes=model.optimize()
model.objective = model.reactions.CrtI
py = product_yield(model.reactions.CrtI, model.reactions.r_1714)
swap_optimization = CofactorSwapOptimization(model=model, cofactor_id_swaps=(["FAD_c","FADH2_c"],["NADP_c","NADPH_c"]),objective_function=py)
solution=swap_optimization.run(max_size=10)

Starting optimization at Fri, 19 Nov 2021 12:15:35


HBox()

Finished after 00:02:58


Below we can see the results of the cofactor swap. It recommends 11 potential swaps that can increase the yield:

In [3]:
solution

,index,targets,fitness
0,0,"(r_0659,)",1.240740
1,1,"(r_0455, r_0547)",1.240740
2,5,"(r_1027, r_0455)",1.182349
3,6,"(r_2140, r_0455)",1.129629
4,8,"(r_0455, r_0481)",1.027389
5,9,"(r_0471,)",0.981427
6,10,"(r_0219,)",0.981427
7,17,"(r_4292,)",0.975810
8,19,"(r_0547,)",0.975810
9,20,"(r_0732,)",0.975810


In [5]:
reactions=solution.data_frame.targets
reactions

0            (r_0659,)
1     (r_0455, r_0547)
2     (r_1027, r_0455)
3     (r_2140, r_0455)
4     (r_0455, r_0481)
5            (r_0471,)
6            (r_0219,)
7            (r_4292,)
8            (r_0547,)
9            (r_0732,)
10           (r_0442,)
Name: targets, dtype: object

Below we can see the reactions recommended in more detail:

In [7]:
for t in reactions:
    print("Reactions:")
    for id in t:
        r=model.reactions.get_by_id(id)
        #print(r.name, id+",","Fitness:", solution.data_frame.fitness[id][10])
        s=""
        for rea in range(len(r.reactants)):
            if rea==len(r.reactants)-1:
                s+=r.reactants[rea].name+" ==> "
            else:
                s+=r.reactants[rea].name+" + "
        for p in range(len(r.products)):
            if p==len(r.products)-1:
                s+=r.products[p].name
            else:
                s+=r.products[p].name+" + "
        print(s)
        print()

Reactions:
isocitrate [cytoplasm] + NADP(+) [cytoplasm] ==> 2-oxoglutarate [cytoplasm] + carbon dioxide [cytoplasm] + NADPH [cytoplasm]

Reactions:
fumarate [cytoplasm] + FADH2 [cytoplasm] ==> H+ [cytoplasm] + succinate [cytoplasm] + FAD [cytoplasm]

H+ [cytoplasm] + L-aspartate 4-semialdehyde [cytoplasm] + NADPH [cytoplasm] ==> L-homoserine [cytoplasm] + NADP(+) [cytoplasm]

Reactions:
H+ [cytoplasm] + sulphite [cytoplasm] + NADPH [cytoplasm] ==> H2O [cytoplasm] + hydrogen sulfide [cytoplasm] + NADP(+) [cytoplasm]

fumarate [cytoplasm] + FADH2 [cytoplasm] ==> H+ [cytoplasm] + succinate [cytoplasm] + FAD [cytoplasm]

Reactions:
acetyl-CoA [cytoplasm] + H+ [cytoplasm] + malonyl-CoA [cytoplasm] + NADPH [cytoplasm] ==> carbon dioxide [cytoplasm] + coenzyme A [cytoplasm] + H2O [cytoplasm] + palmitoyl-CoA [cytoplasm] + NADP(+) [cytoplasm]

fumarate [cytoplasm] + FADH2 [cytoplasm] ==> H+ [cytoplasm] + succinate [cytoplasm] + FAD [cytoplasm]

Reactions:
fumarate [cytoplasm] + FADH2 [cytoplasm

If the co-factors are swapped in the reactions listed above, it could lead to an increase in the lycopene yield. Since the lycopene pathway used 4 FAD, we expect that co-factors can have a substantial effect on the yield. The fitness can be observed to go above 1, which would indicate that more than 1 lycopene can be created from 1 glucose molecule, however, this is impossible.